In [1]:
#packages used
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def convert_to_float(column):
    new=[]
    for f in range(len(column)):
        new.append(float(column[f].replace("$","").replace(",","").replace("%","")))
    return new    

# Part 1: Data Cleaning, Filtering and Parsing

### 1a. California DMV Vechicle Registration by Type (y variable)

#### Problems/ things done:

<ol>
  <li>Select columns Battery Electric, Plug-in Hybrid, and Grand Total.</li>

  <li>Turn columns into floats by turning 100,003 into 100003.0</li>
  
  <li>Get combined number of electric vehicles by adding  Plug in Hybrid and Battery Electric</li>
  
   <li>Make a new dataframe with columns 'Total EV's', 'Total Registered Vehicles', and 'County'.</li>

</ol>

In [3]:
ev_raw=pd.read_csv('California_DMV_EV.csv')
ev_raw.head()

,County,Battery Electric,Diesel,Diesel Hybrid,Ethanol,Fuel Cell,Gasoline,Hybrid Gas,Plug-in Hybrid,Butane,natural gas,Methanol,Methane,Natural Gas,Propane,Grand Total
0,Alameda,"14,904","38,103",36,"53,198",170,"1,048,019","62,088","10,342",1,91,195,0,"1,173",56,"1,228,376"
1,Alpine,2,177,0,92,0,"1,570",30,1,0,0,0,0,0,0,"1,872"
2,Amador,51,"4,319",0,"2,534",0,"36,766",753,58,0,0,7,0,12,13,"44,513"
3,Butte,238,"13,287",0,"11,289",0,"153,132","4,107",245,0,15,48,0,49,13,"182,423"
4,Calaveras,75,"6,109",0,"3,242",0,"47,294",990,81,0,0,14,0,8,3,"57,816"


In [4]:
y_data=ev_raw.loc[:,['County ','Battery Electric ','Plug-in Hybrid ','Grand Total ']]

def to_float(column):
    new=[]
    for k in range(len(column)):
        new.append(float(column[k].replace(",","")))
    return new

battery=to_float(np.array(y_data['Battery Electric ']))
hybrid=to_float(np.array(y_data['Plug-in Hybrid ']))

total=np.array(battery)+np.array(hybrid)

total_evs=pd.DataFrame()
total_evs['Total Vehicles']=to_float(y_data['Grand Total '])
total_evs['County']=y_data['County ']
total_evs['EV']=total

<h3 align="center">1a. Output</h3>

In [5]:
total_evs.head()

print(len(total_evs['County'][0]))

8


## 1b. California County Charging Station

#### Problems/ things done:

<ol>
  <li>Fix spelling/capitalize/add space to the City column for our charging station data. (Sanfracisco --> San Francisco)</li>

  <li>Map city to a county by using a dataframe that has all cities and associated counties in California</li>
  
  <li>Create a grouped dataframe that has total count of charging stations by county</li>
  
   <li>Include a list of cities that have no link to county, and get a count of how much data was 'lost'.</li>
</ol>

In [6]:
stations=pd.read_csv('charging_stations.csv') #raw data
stations.head(2)

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Connector Types,Country,Intersection Directions (French),Access Days Time (French),BD Blends (French),Groups With Access Code (French),Hydrogen Is Retail,Access Code,Access Detail Code,Federal Agency Code
0,ELEC,LADWP - Truesdale Center,11791 Truesdale St,NaN,Sun Valley,CA,91352,NaN,NaN,E,...,J1772,US,NaN,NaN,NaN,Privé - Réservé au gouvernement,NaN,private,GOVERNMENT,NaN
1,ELEC,Southern California Edison - Rosemead Office B...,2244 Walnut Grove Ave,NaN,Rosemead,CA,91770,NaN,NaN,E,...,J1772 NEMA520,US,NaN,NaN,NaN,Privé,NaN,private,NaN,NaN


In [7]:
cnty=pd.read_csv('county_city.csv').drop(['Type','Land area[1]',
                                          'Incorporated[7]',
                                          'Population (2010)[1][8][9]',
                                         'Land area[1].1'],axis=1).drop(0,axis=0)

cnty.head(5)
#this has the link between city ane county. Contains all cities and their associated counties. Well use these two data
#frames to get a count of charging stations by county.

,Name,County
1,Adelanto,San Bernardino
2,Agoura Hills,Los Angeles
3,Alameda,Alameda
4,Albany,Alameda
5,Alhambra,Los Angeles


In [8]:
station_city=stations['City'] #pulling all cities from the station dataframe.
print(station_city[4]) #well be interating over this to get a county label from this printed city/town label

new_cities=[]
for city in station_city:
    new_cities.append((city.title().strip(',')))
#Some values are not capitlized or have commas. This undues human error

Los Angeles


In [9]:
county_names=[] #end result is a list of the associated county names


for i in range(len(new_cities)):
    city=stations['City'][i]  #select the city string
    
    
    result=cnty[ cnty['Name'] == city] #link it to the dataframe where we know cities to counties
    
    
    if result.shape[0]==1: #if we get a result, get the county label and append it 
        county_found=np.array(result['County'])[0]

        county_names.append(county_found)
        
        
    else: #if we dont get a result (the city input has no county because it was mispelled or not written correctly,
                                    # lets append a 0 to the county name so we can keep track of the lost cities)
        county_names.append(0) 
        
print(county_names[:5] ) #end result.

[0, 'Los Angeles', 'Los Angeles', 'Los Angeles', 'Los Angeles']


In [10]:
charging_stations_filtered=pd.DataFrame()
charging_stations_filtered['County']=county_names
charging_stations_filtered['City']=stations['City']
display(charging_stations_filtered.head(5))

print('this dataframe will now be grouped by county, counts')

,County,City
0,0,Sun Valley
1,Los Angeles,Rosemead
2,Los Angeles,Los Angeles
3,Los Angeles,Los Angeles
4,Los Angeles,Los Angeles


this dataframe will now be grouped by county, counts


In [30]:
#lets make a count for how many charging stations each county has

stations_by_county=charging_stations_filtered.groupby('County').aggregate(len)  #len does the same as count
stations_by_county.rename(index=str,columns={'City':'Charging Stations'},inplace=True)
display(stations_by_county.head())
print(stations_by_county.shape,'This dataframe has 49 counties. Which means some 9 counties have no data, due to not having any charing stations or mispelled data')

#our data says it detected a nonzero amount of charging stations for 49 counties, which means we have 9 counties that
#essentially are 0. In addition, we lost 10% of our data due to mislabeled cities.

,Charging Stations
County,
0,469
Alameda,262
Amador,10
Butte,14
Colusa,1


(50, 1) This dataframe has 49 counties. Which means some 9 counties have no data, due to not having any charing stations or mispelled data


In [12]:
lost_data=charging_stations_filtered[charging_stations_filtered['County']==0]
display(lost_data.dropna().head())

total_amount=sum(stations_by_county['City'])
print('The total_amount of charging stations is',total_amount,'The toal amount of lost cities is',lost_data.shape[0])
print('We lost',lost_data.shape[0]/total_amount,'of data')
#for viewing purposes to see which cities were lost. Some were written as S.F. others are neighborhoods, others are 
#names of colleges.

,County,City
0,0,Sun Valley
8,0,Wilmington
9,0,Sylmar
16,0,La Jolla
19,0,Sepulveda


The total_amount of charging stations is 5392 The toal amount of lost cities is 469
We lost 0.0869807121661721 of data


<h3 align="center">1b. Output</h3>

In [13]:
stations_by_county.head()

,City
County,
0,469
Alameda,262
Amador,10
Butte,14
Colusa,1


### 1C. California politics by county

#### Problems/ things done:

<ol>
  <li>Removed raw number of democrats/repub etc and kept percentages</li>
    
  <li>Kept only Democratic, Republican, and Green party percentage identification</li>

  <li>Relabeled the percentage column to the correct county</li>
  
  <li>Turn all strings and percentages into floats</li>
  
</ol>

In [14]:
raw_politics=pd.read_csv('county_politics.csv')       #make sure county_politics is in your directory
raw_politics.drop('Independent',axis=1,inplace=True)  #independent not seemed important
raw_politics.head()

#problem to fix: County information comes in row pairs. The first row is raw number of people and the 2nd has
#percentages. We want only percentages and we want it to be labeled by its appr county instead of 'percent'

,County,Eligible,Registered,Democratic,Republican,Green
0,Alameda,"1,088,060","853,629","479,107","96,400","6,640"
1,Percent,NaN,78.45%,56.13%,11.29%,0.78%
2,Alpine,939,724,294,202,6
3,Percent,NaN,77.10%,40.61%,27.90%,0.83%
4,Amador,"27,045","21,763","6,327","9,652",103


In [15]:
county_names=raw_politics['County']

new_county_names=[] #this will be the new labels 

for value in county_names[0::2]:    #the [0::2] only picks out even integer values in the array of county names
    new_county_names+=[value,value] #well go from ['Alameda,per,Alpine,per']--->['Alameda','Alameda','Alpine',Alpine']
    
raw_politics['County']=new_county_names  #now they are properly labeled, but we still have info we dont need

In [16]:
final_politics=raw_politics.loc[1::2,:] #this removes the values we dont need, which is the raw counts. [1::2] only 
                                        #picks out even values in the df
    
final_politics.index=np.arange(0,final_politics.shape[0])  #make the indexs start from 0,1,2,3,4...

final_politics.drop('Eligible',axis=1,inplace=True) #dont need eligible

/Users/nathancarrizales-duarte/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [17]:
county_politics=final_politics['County']
final_politics=final_politics.drop('County',axis=1).apply(convert_to_float)
final_politics['County']=county_politics

<h3 align="center">1c. Output</h3>

In [18]:
final_politics.head()

,Registered,Democratic,Republican,Green,County
0,78.45,56.13,11.29,0.78,Alameda
1,77.10,40.61,27.90,0.83,Alpine
2,80.47,29.07,44.35,0.47,Amador
3,67.50,34.57,35.21,0.80,Butte
4,79.45,27.84,41.63,0.67,Calaveras


### 1d. California county statistics by US. Census

#### Problems/ things done:

<ol>
  <li>Merge together 12 dataframes, corresponding to 12 csv files, 1-11 containing 5 counties and 12 containing 3 counties </li>

  <li>Fix the column names to correspond with features</li>
  
  <li>Rename County column strings, such as 'Alameda County, California', to 'Alameda County'. </li>
  
   <li>Select a subset of features that will be used for the model</li>
</ol>

In [19]:
df1=pd.read_csv('group1.csv').iloc[:,[0,2,4,6,8,10]]
df1.head()

,Fact,"Calaveras County, California","Butte County, California","Amador County, California","Alpine County, California","Alameda County, California"
0,"Population estimates, July 1, 2017, (V2017)","45,670","229,294","38,626","1,120","1,663,190"
1,"Population estimates base, April 1, 2010, (V2...","45,578","220,002","38,091","1,175","1,510,261"
2,"Population, percent change - April 1, 2010 (es...",0.2%,4.2%,1.4%,-4.7%,10.1%
3,"Population, Census, April 1, 2010","45,578","220,000","38,091","1,175","1,510,271"
4,"Persons under 5 years, percent",4.3%,5.5%,4.1%,3.0%,5.9%


In [20]:
df2=pd.read_csv('group2.csv').iloc[:,[0,2,4,6,8,10]]
df2.head()

df3=pd.read_csv('group3.csv').iloc[:,[0,2,4,6,8,10]]
df3.head()

df4=pd.read_csv('group4.csv').iloc[:,[0,2,4,6,8,10]]
df4.head()

df5=pd.read_csv('group5.csv').iloc[:,[0,2,4,6,8,10]]
df5.head()

df6=pd.read_csv('group6.csv').iloc[:,[0,2,4,6,8,10]]
df6.head()

df7=pd.read_csv('group7.csv').iloc[:,[0,2,4,6,8,10]]
df7.head()

df8=pd.read_csv('group8.csv').iloc[:,[0,2,4,6,8,10]]
df8.head()

df9=pd.read_csv('group9.csv').iloc[:,[0,2,4,6,8,10]]
df9.head()

df10=pd.read_csv('group10.csv').iloc[:,[0,2,4,6,8,10]]
df10.head()

df11=pd.read_csv('group11.csv').iloc[:,[0,2,4,6,8,10]]
df11.head()

df12=pd.read_csv('group12.csv').iloc[:,[0,2,4,6]]
df12.head()

,Fact,"Yuba County, California","Yolo County, California","Ventura County, California"
0,"Population estimates, July 1, 2017, (V2017)","77,031","219,116","854,223"
1,"Population estimates base, April 1, 2010, (V2...","72,146","200,850","823,391"
2,"Population, percent change - April 1, 2010 (es...",6.8%,9.1%,3.7%
3,"Population, Census, April 1, 2010","72,155","200,849","823,318"
4,"Persons under 5 years, percent",7.8%,5.7%,6.0%


In [21]:
#making one big dataframe with all states
df_12=pd.merge(df1,df2,on='Fact')

df_34=pd.merge(df3,df4,on='Fact')

df_56=pd.merge(df5,df6,on='Fact')

df_78=pd.merge(df7,df8,on='Fact')

df_910=pd.merge(df9,df10,on='Fact')

df_1112=pd.merge(df11,df12,on='Fact')

df_1234=pd.merge(df_12,df_34,on='Fact')

df_5678=pd.merge(df_56,df_78,on='Fact')

df_9101112=pd.merge(df_910,df_1112,on='Fact').iloc[:50]


df_12345678=pd.merge(df_1234,df_5678,on='Fact').iloc[:50]


df_all=pd.merge(df_12345678,df_9101112,on='Fact').iloc[0:50,:].T  # transpose because i want counties on the rows

In [22]:
#fixes column labels
df_all.columns=df_all.loc['Fact']
df_all.drop("Fact",inplace=True)


df_all[df_all.columns[0]]=df_all.index  #replaces array data of numbers with strings of state names
df_all.rename(index=str, columns={df_all.columns[0]: "County"},inplace=True) #relabels it to state
df_all.head()

#Lastly, this sorts the states by name so it looks nicer, then makes alabama index 0
df_all.sort_values('County',ascending=True,inplace=True)
df_all.index=np.arange(len(df_all.index))

In [23]:
features=[0,1,4,5,6,8,9,10,11,12,13,14,15,16,18,23,25,26,28,29,30,41,42,44]
for i in features:
    print(df_all.columns[i])

County
Population estimates base, April 1, 2010,  (V2017)
Persons under 5 years, percent
Persons under 18 years, percent
Persons 65 years and over, percent
White alone, percent
Black or African American alone, percent
American Indian and Alaska Native alone, percent
Asian alone, percent
Native Hawaiian and Other Pacific Islander alone, percent
Two or More Races, percent
Hispanic or Latino, percent
White alone, not Hispanic or Latino, percent
Veterans, 2012-2016
Housing units,  July 1, 2017,  (V2017)
Median gross rent, 2012-2016
Households, 2012-2016
Persons per household, 2012-2016
Language other than English spoken at home, percent of persons age 5 years+, 2012-2016
High school graduate or higher, percent of persons age 25 years+, 2012-2016
Bachelor's degree or higher, percent of persons age 25 years+, 2012-2016
Mean travel time to work (minutes), workers age 16 years+, 2012-2016
Median household income (in 2016 dollars), 2012-2016
Persons in poverty, percent


In [24]:
us_cen_county=df_all.iloc[:,features]


fixed_county_names=[]
for i in us_cen_county['County']:
    fixed_county_names.append(i.replace('County, California','').replace('city, California',''))

    

us_cen_county['County']=fixed_county_names

/Users/nathancarrizales-duarte/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


<h3 align="center">1d. Output</h3>

In [25]:
us_cen_county.head()

Fact,County,"Population estimates base, April 1, 2010, (V2017)","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent","White alone, percent","Black or African American alone, percent","American Indian and Alaska Native alone, percent","Asian alone, percent","Native Hawaiian and Other Pacific Islander alone, percent",...,"Housing units, July 1, 2017, (V2017)","Median gross rent, 2012-2016","Households, 2012-2016","Persons per household, 2012-2016","Language other than English spoken at home, percent of persons age 5 years+, 2012-2016","High school graduate or higher, percent of persons age 25 years+, 2012-2016","Bachelor's degree or higher, percent of persons age 25 years+, 2012-2016","Mean travel time to work (minutes), workers age 16 years+, 2012-2016","Median household income (in 2016 dollars), 2012-2016","Persons in poverty, percent"
0,Alameda,"1,510,261",5.9%,20.7%,13.5%,50.2%,11.3%,1.1%,31.1%,1.0%,...,"606,052","$1,432","564,293",2.79,43.9%,87.3%,43.9%,31.6,"$79,831",10.7%
1,Alpine,"1,175",3.0%,17.2%,23.2%,71.6%,0.4%,22.4%,2.0%,0.1%,...,"1,771","$1,109",343,3.34,10.2%,91.1%,29.7%,22.4,"$62,375",18.6%
2,Amador,"38,091",4.1%,15.2%,26.6%,89.8%,2.5%,2.3%,1.7%,0.3%,...,"18,317","$1,061","14,248",2.31,11.4%,88.3%,21.5%,31.2,"$57,032",11.3%
3,Butte,"220,002",5.5%,20.0%,18.2%,86.1%,1.8%,2.5%,4.8%,0.3%,...,"99,404",$936,"85,505",2.55,14.2%,88.3%,26.1%,19.8,"$44,366",19.5%
4,Calaveras,"45,578",4.3%,17.0%,26.9%,91.5%,0.9%,1.9%,1.7%,0.3%,...,"28,429","$1,120","17,713",2.50,7.0%,90.4%,20.2%,34.9,"$53,502",13.1%


### 1e. California Income by County

#### Problems/ things done:

<ol>
  <li>Select only county and median income columns</li>

  <li>Convert values such as 100,442$ to floats</li>
  
  <li>Reset index</li>
  
</ol>

In [26]:
income=pd.read_csv('County_income.csv')#.iloc[:,['Median Income ']]
income.loc[:,['County','Median Income']]



income['Median Income']=convert_to_float(income['Median Income'])
income=income.loc[:,['County','Median Income']].sort_values('County').reset_index().drop('index',axis=1)

<h3 align="center">1e. Output</h3>

In [27]:
income.head()

,County,Median Income
0,Alameda,50031.0
1,Alpine,40869.0
2,Amador,39090.0
3,Butte,32261.0
4,Calaveras,38936.0


# Part 2: Merging

#### Problems/ things done:

<ol>
    
    
   <li>Merge 5 dataframes together.</li>
  <li>Fixed the total_evs county labels, because there is a hidden white space after the end of every county name ('Alameda_' will be turned into 'Alameda') </li>

  <li>Fix the us_cen_county county string names because there are some minute discrepancies against the other dataframes</li>
  
  
</ol>

In [34]:
total_evs['County']=final_politics['County']

merge1=pd.merge(total_evs,final_politics,on='County')
display(merge1.head())
merge1.shape[0]==58 #need to have 58 points

,Total Vehicles,County,EV,Registered,Democratic,Republican,Green
0,1228376.0,Alameda,25246.0,78.45,56.13,11.29,0.78
1,1872.0,Alpine,3.0,77.10,40.61,27.90,0.83
2,44513.0,Amador,109.0,80.47,29.07,44.35,0.47
3,182423.0,Butte,483.0,67.50,34.57,35.21,0.80
4,57816.0,Calaveras,156.0,79.45,27.84,41.63,0.67


True

In [35]:
merge2=pd.merge(stations_by_county,income,on='County',how='right').fillna(0) #10 counties didnt have any charging stations
display(merge2.head())

merge2.shape[0]==58 #need to have 58 points

,County,Charging Stations,Median Income
0,Alameda,262.0,50031.0
1,Amador,10.0,39090.0
2,Butte,14.0,32261.0
3,Colusa,1.0,33799.0
4,Contra Costa,137.0,50667.0


True

In [37]:
merge3=pd.merge(merge1,merge2,on='County')
display(merge3.head())

merge3.shape[0]==58 #need to have 58 points

,Total Vehicles,County,EV,Registered,Democratic,Republican,Green,Charging Stations,Median Income
0,1228376.0,Alameda,25246.0,78.45,56.13,11.29,0.78,262.0,50031.0
1,1872.0,Alpine,3.0,77.10,40.61,27.90,0.83,0.0,40869.0
2,44513.0,Amador,109.0,80.47,29.07,44.35,0.47,10.0,39090.0
3,182423.0,Butte,483.0,67.50,34.57,35.21,0.80,14.0,32261.0
4,57816.0,Calaveras,156.0,79.45,27.84,41.63,0.67,0.0,38936.0


True

In [43]:
us_cen_county['County']=merge3['County']
all_var=pd.merge(us_cen_county,merge3,on='County')
all_var.head()

all_var.shape[0]==58 #need to have 58 points

/Users/nathancarrizales-duarte/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


True

<h3 align="center">2a. Output</h3>

In [46]:
all_var.head()

,County,"Population estimates base, April 1, 2010, (V2017)","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent","White alone, percent","Black or African American alone, percent","American Indian and Alaska Native alone, percent","Asian alone, percent","Native Hawaiian and Other Pacific Islander alone, percent",...,"Median household income (in 2016 dollars), 2012-2016","Persons in poverty, percent",Total Vehicles,EV,Registered,Democratic,Republican,Green,Charging Stations,Median Income
0,Alameda,"1,510,261",5.9%,20.7%,13.5%,50.2%,11.3%,1.1%,31.1%,1.0%,...,"$79,831",10.7%,1228376.0,25246.0,78.45,56.13,11.29,0.78,262.0,50031.0
1,Alpine,"1,175",3.0%,17.2%,23.2%,71.6%,0.4%,22.4%,2.0%,0.1%,...,"$62,375",18.6%,1872.0,3.0,77.10,40.61,27.90,0.83,0.0,40869.0
2,Amador,"38,091",4.1%,15.2%,26.6%,89.8%,2.5%,2.3%,1.7%,0.3%,...,"$57,032",11.3%,44513.0,109.0,80.47,29.07,44.35,0.47,10.0,39090.0
3,Butte,"220,002",5.5%,20.0%,18.2%,86.1%,1.8%,2.5%,4.8%,0.3%,...,"$44,366",19.5%,182423.0,483.0,67.50,34.57,35.21,0.80,14.0,32261.0
4,Calaveras,"45,578",4.3%,17.0%,26.9%,91.5%,0.9%,1.9%,1.7%,0.3%,...,"$53,502",13.1%,57816.0,156.0,79.45,27.84,41.63,0.67,0.0,38936.0


# Dataframes to be cleaned/merged

In [49]:
real_per_cap=pd.read_csv('real_cap.csv')
real_per_cap.head()

,County,Real Per Capita Income (dollars)
0,Alameda,67837
1,Alpine,76215
2,Amador,45804
3,Butte,43381
4,Calaveras,49341


In [56]:
gas_price=pd.read_csv('gas_price.csv').drop(['Unnamed: 3','Unnamed: 4'],axis=1)
gas_price.head()

,County,2018 Price,2030 Price
0,Alameda,3.655,$6.68
1,Alpine,3.624,$6.62
2,Amador,3.542,$6.47
3,Butte,3.45,$6.30
4,Calaveras,3.476,$6.35


In [58]:
unemp=pd.read_csv('unemploy_2018.csv')
unemp.head()

,County,Sep-18
0,Alameda County,2.8
1,Alpine County,4.9
2,Amador County,3.4
3,Butte County,4.1
4,Calaveras County,3.5
